# Importing required libraries

In [1]:
import matplotlib
import pandas as pd
import numpy as np
import math
from numpy import argmax
import sys
from IPython.display import display
import warnings
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_absolute_percentage_error

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Concatenate

import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

from tensorflow.keras.utils import plot_model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import callbacks

from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
import numpy as np
import time
import statistics

# library options
%matplotlib inline
pd.options.display.max_columns = None
warnings.filterwarnings('ignore')

from keras import backend
 
def rmse(y_true, y_pred):
    return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))

In [2]:
# Monte Carlo Dropout Layer
class MonteCarloDropout(keras.layers.Dropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

# Read in the Train and Test data

In [3]:
X_train = np.array(pd.read_parquet('data/X_train.parquet')).astype('float')
X_test = np.array(pd.read_parquet('data/X_test.parquet')).astype('float')

y_test_class = np.array(pd.read_parquet('data/y_test.parquet')['sum_payments_package_key_ltv_class'])
y_train_class = np.array(pd.read_parquet('data/y_train.parquet')['sum_payments_package_key_ltv_class'])                        
#y_test_class = np.array(pd.read_parquet('data/y_test.parquet')['sum_payments_package_key_ltv_class'].apply(lambda x: 0 if x == 'low' else 1 if x == "medium" else 2 if x == "high" else -1)).astype('float')
#y_train_class = np.array(pd.read_parquet('data/y_test.parquet')['sum_payments_package_key_ltv_class'].apply(lambda x: 0 if x == 'low' else 1 if x == "medium" else 2 if x == "high" else -1)).astype('float')
y_test_reg = np.array(pd.read_parquet('data/y_test.parquet')['sum_payments_package_key_ltv']).astype('float')
y_train_reg = np.array(pd.read_parquet('data/y_train.parquet')['sum_payments_package_key_ltv']).astype('float')

# Check Data before Modelling 

In [4]:
print('Training Features Shape:', X_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Training Labels Regression Shape:', y_train_reg.shape)
print('Testing Labels Regression Shape:', y_test_reg.shape)

Training Features Shape: (111791, 180)
Testing Features Shape: (27948, 180)
Training Labels Regression Shape: (111791,)
Testing Labels Regression Shape: (27948,)


# Hyperparameter Tuning

In [5]:
def model_builder(hp):
    model = keras.Sequential()

    # layer 01
    layer_01 = hp.Choice('layer_01', values = ['False','True'])
    if layer_01:
        units_01 = hp.Int('units_01', min_value=64, max_value=1024, step=32)
        activation_01 = hp.Choice('activation_01', values = ['LeakyReLU','relu','sigmoid','tanh'])
        kernel_initializer_01 = hp.Choice('kernel_01', values = ['he_uniform', 'he_normal', 'glorot_uniform', 'glorot_normal'])
        model.add(Dense(units_01, activation = activation_01, kernel_initializer=kernel_initializer_01, input_shape = (X_train.shape[1],)))
    
    batch_01 = hp.Choice('batch_01', values = ['False','True'])
    if batch_01:
        model.add(BatchNormalization())
    
    drop_01 = hp.Choice('drop_01', values = ['False','drop','Monte'])
    if drop_01 == 'drop':
        dropout_01 = hp.Float('dropout_01', min_value=0.1, max_value=0.6, step=0.1)
        model.add(Dropout(dropout_01))
    elif drop_01 == 'Monte':
        monte_01 = hp.Float('monte_01', min_value=0.1, max_value=0.6, step=0.1)
        model.add(MonteCarloDropout(monte_01))
    
    # layer 02
    layer_02 = hp.Choice('layer_02', values = ['False','True'])
    if layer_02:
        units_02 = hp.Int('units_02', min_value = 32, max_value = 512, step=32)
        activation_02 = hp.Choice('activation_02', values = ['LeakyReLU','relu','sigmoid','tanh'])
        kernel_initializer_02 = hp.Choice('kernel_02', values = ['he_uniform', 'he_normal', 'glorot_uniform', 'glorot_normal'])
        model.add(Dense(units_02, activation = activation_02, kernel_initializer=kernel_initializer_02))
    
    batch_02 = hp.Choice('batch_02', values = ['False','True'])
    if batch_02:
        model.add(BatchNormalization())
        
    drop_02 = hp.Choice('drop_02', values = ['False','drop','Monte'])
    if drop_02:
        dropout_02 = hp.Float('dropout_02', min_value=0.1, max_value=0.6, step=0.1)
        model.add(Dropout(dropout_02))
    elif drop_02 == 'Monte':
        monte_02 = hp.Float('monte_02', min_value=0.1, max_value=0.6, step=0.1)
        model.add(MonteCarloDropout(monte_02))
    
    # layer 03
    layer_03 = hp.Choice('layer_03', values = ['False','True'])
    if layer_03:
        units_03 = hp.Int('units_03', min_value = 32, max_value = 512, step=32)
        activation_03 = hp.Choice('activation_03', values = ['LeakyReLU','relu','sigmoid','tanh'])
        kernel_initializer_03 = hp.Choice('kernel_03', values = ['he_uniform', 'he_normal', 'glorot_uniform', 'glorot_normal'])
        model.add(Dense(units_03, activation = activation_03, kernel_initializer=kernel_initializer_03))
    
    drop_03 = hp.Choice('drop_03', values = ['False','drop','Monte'])
    if drop_03:
        dropout_03 = hp.Float('dropout_03', min_value=0.1, max_value=0.6, step=0.1)
        model.add(Dropout(dropout_03))
    elif drop_03 == 'Monte':
        monte_03 = hp.Float('monte_03', min_value=0.1, max_value=0.6, step=0.1)
        model.add(MonteCarloDropout(monte_03))
    
    batch_03 = hp.Choice('batch_03', values = ['False','True'])
    if batch_03:
        model.add(BatchNormalization())
    
    # layer 04
    layer_04 = hp.Choice('layer_04', values = ['False','True'])
    if layer_04:
        units_04 = hp.Int('units_04', min_value = 16, max_value = 512, step=32)
        activation_04 = hp.Choice('activation_04', values = ['LeakyReLU','relu','sigmoid','tanh'])
        kernel_initializer_04 = hp.Choice('kernel_04', values = ['he_uniform', 'he_normal', 'glorot_uniform', 'glorot_normal'])
        model.add(Dense(units_04, activation = activation_04, kernel_initializer=kernel_initializer_04))
    
    drop_04 = hp.Choice('drop_04', values = ['False','drop','Monte'])
    if drop_04:
        dropout_04 = hp.Float('dropout_04', min_value=0.1, max_value=0.6, step=0.1)
        model.add(Dropout(dropout_04))
    elif drop_04 == 'Monte':
        monte_04 = hp.Float('monte_04', min_value=0.1, max_value=0.6, step=0.1)
        model.add(MonteCarloDropout(monte_04))
        
    batch_04 = hp.Choice('batch_04', values = ['False','True'])
    if batch_04:
        model.add(BatchNormalization())
    
    # layer 05
    layer_05 = hp.Choice('layer_05', values = ['False','True'])
    if layer_05:
        units_05 = hp.Int('units_05', min_value = 16, max_value = 256, step=32)
        activation_05 = hp.Choice('activation_05', values = ['LeakyReLU','relu','sigmoid','tanh'])
        kernel_initializer_05 = hp.Choice('kernel_05', values = ['he_uniform', 'he_normal', 'glorot_uniform', 'glorot_normal'])
        model.add(Dense(units_05, activation = activation_05, kernel_initializer=kernel_initializer_05))
    
    drop_05 = hp.Choice('drop_05', values = ['False','drop','Monte'])
    if drop_05:
        dropout_05 = hp.Float('dropout_05', min_value=0.1, max_value=0.6, step=0.1)
        model.add(Dropout(dropout_05))
    elif drop_05 == 'Monte':
        monte_05 = hp.Float('monte_05', min_value=0.1, max_value=0.6, step=0.1)
        model.add(MonteCarloDropout(monte_05))
        
    batch_05 = hp.Choice('batch_05', values = ['False','True'])
    if batch_05:
        model.add(BatchNormalization())
    
    # layer 06
    layer_06 = hp.Choice('layer_06', values = ['False','True'])
    if layer_06:
        units_06 = hp.Int('units_06', min_value = 4, max_value = 16, step=4)
        activation_06 = hp.Choice('activation_06', values = ['LeakyReLU','relu','sigmoid','tanh'])
        kernel_initializer_06 = hp.Choice('kernel_06', values = ['he_uniform', 'he_normal', 'glorot_uniform', 'glorot_normal'])
        model.add(Dense(units_06, activation = activation_06, kernel_initializer=kernel_initializer_06))
        
    # layer 07
    layer_07 = hp.Choice('layer_07', values = ['False','True'])
    if layer_07:
        units_07 = hp.Int('units_07', min_value = 4, max_value = 16, step=4)
        activation_07 = hp.Choice('activation_07', values = ['LeakyReLU','relu','sigmoid','tanh'])
        kernel_initializer_07 = hp.Choice('kernel_07', values = ['he_uniform', 'he_normal', 'glorot_uniform', 'glorot_normal'])
        model.add(Dense(units_07, activation = activation_07, kernel_initializer=kernel_initializer_07))    
    
    # layer 08
    layer_08 = hp.Choice('layer_08', values = ['False','True'])
    if layer_08:
        units_08 = hp.Int('units_08', min_value = 4, max_value = 16, step=4)
        activation_08 = hp.Choice('activation_08', values = ['LeakyReLU','relu','sigmoid','tanh'])
        kernel_initializer_08 = hp.Choice('kernel_08', values = ['he_uniform', 'he_normal', 'glorot_uniform', 'glorot_normal'])
        model.add(Dense(units_08, activation = activation_08, kernel_initializer=kernel_initializer_08))
        
    # layer 09
    layer_09 = hp.Choice('layer_06', values = ['False','True'])
    if layer_09:
        units_09 = hp.Int('units_06', min_value = 4, max_value = 16, step=4)
        activation_09 = hp.Choice('activation_09', values = ['LeakyReLU','relu','sigmoid','tanh'])
        kernel_initializer_09 = hp.Choice('kernel_09', values = ['he_uniform', 'he_normal', 'glorot_uniform', 'glorot_normal'])
        model.add(Dense(units_09, activation = activation_09, kernel_initializer=kernel_initializer_09))
        
    # layer 10
    layer_10 = hp.Choice('layer_10', values = ['False','True'])
    if layer_10:
        units_10 = hp.Int('units_10', min_value = 4, max_value = 16, step=4)
        activation_10 = hp.Choice('activation_10', values = ['LeakyReLU','relu','sigmoid','tanh'])
        kernel_initializer_10 = hp.Choice('kernel_10', values = ['he_uniform', 'he_normal', 'glorot_uniform', 'glorot_normal'])
        model.add(Dense(units_10, activation = activation_10, kernel_initializer=kernel_initializer_10))
    
    model.add(Dense(1))
    
    # Tune the learning rate for the optimizer
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer = keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss = 'mse', metrics = ['mse', 'mae',rmse,'mape','msle'])

    return model

In [6]:
# Select Tuner Hyperband
tuner = kt.Hyperband(model_builder,
                     objective = 'mse',
                     max_epochs = 40,
                     factor = 6,
                     project_name = 'MLP_Regression_02')

stop_early = callbacks.EarlyStopping(monitor='loss', patience=7)

INFO:tensorflow:Reloading Oracle from existing project ./MLP_Regression_02/oracle.json
INFO:tensorflow:Reloading Tuner from ./MLP_Regression_02/tuner0.json


In [7]:
tuner.search(X_train, y_train_reg, epochs=100, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print('The hyperparameter search is complete. The optimal model is:')
print(best_hps)

INFO:tensorflow:Oracle triggered exit
The hyperparameter search is complete. The optimal model is:


In [8]:
tuner.results_summary()

Results summary
Results in ./MLP_Regression_02
Showing 10 best trials
Objective(name='mse', direction='min')
Trial summary
Hyperparameters:
layer_01: False
units_01: 992
activation_01: tanh
kernel_01: he_uniform
batch_01: False
drop_01: drop
layer_02: True
units_02: 128
activation_02: LeakyReLU
kernel_02: he_uniform
batch_02: True
drop_02: drop
dropout_02: 0.4
layer_03: True
units_03: 256
activation_03: sigmoid
kernel_03: glorot_normal
drop_03: drop
dropout_03: 0.30000000000000004
batch_03: False
layer_04: False
units_04: 240
activation_04: relu
kernel_04: glorot_normal
drop_04: False
dropout_04: 0.6
batch_04: False
layer_05: False
units_05: 144
activation_05: relu
kernel_05: glorot_uniform
drop_05: False
dropout_05: 0.6
batch_05: True
layer_06: True
units_06: 12
activation_06: relu
kernel_06: glorot_normal
layer_07: True
units_07: 8
activation_07: tanh
kernel_07: glorot_normal
layer_08: False
units_08: 4
activation_08: sigmoid
kernel_08: he_normal
activation_09: relu
kernel_09: he_uni

activation_06: tanh
kernel_06: glorot_normal
layer_07: True
units_07: 16
activation_07: tanh
kernel_07: glorot_uniform
layer_08: False
units_08: 8
activation_08: tanh
kernel_08: he_uniform
activation_09: sigmoid
kernel_09: glorot_normal
layer_10: True
units_10: 8
activation_10: relu
kernel_10: he_normal
learning_rate: 0.001
dropout_01: 0.1
monte_01: 0.2
tuner/epochs: 7
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 762.2202758789062
Trial summary
Hyperparameters:
layer_01: True
units_01: 256
activation_01: relu
kernel_01: he_uniform
batch_01: False
drop_01: drop
layer_02: True
units_02: 192
activation_02: sigmoid
kernel_02: he_normal
batch_02: True
drop_02: Monte
dropout_02: 0.30000000000000004
layer_03: True
units_03: 416
activation_03: sigmoid
kernel_03: he_uniform
drop_03: drop
dropout_03: 0.30000000000000004
batch_03: False
layer_04: True
units_04: 496
activation_04: tanh
kernel_04: glorot_normal
drop_04: False
dropout_04: 0.4
batch_04: True
layer_05: True
units_05: 

In [ ]:
# Build the model with the optimal hyperparameters and train it on the data for 100 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train_reg,validation_split=0.33, epochs = 100)

Epoch 1/100
2341/2341 [==============================] - 18s 7ms/step - loss: 1201.5404 - mse: 1201.5404 - mae: 18.9991 - rmse: 18.9991 - mape: 231.5899 - msle: 1.4051 - val_loss: 828.5510 - val_mse: 828.5510 - val_mae: 15.6648 - val_rmse: 15.6648 - val_mape: 166.6517 - val_msle: 0.7993
Epoch 2/100
2341/2341 [==============================] - 15s 6ms/step - loss: 846.0843 - mse: 846.0843 - mae: 16.4206 - rmse: 16.4206 - mape: 214.2570 - msle: 0.9619 - val_loss: 782.6606 - val_mse: 782.6606 - val_mae: 16.3411 - val_rmse: 16.3411 - val_mape: 227.3297 - val_msle: 0.9841
Epoch 3/100
2341/2341 [==============================] - 14s 6ms/step - loss: 808.6876 - mse: 808.6876 - mae: 15.9428 - rmse: 15.9428 - mape: 208.8731 - msle: 0.9393 - val_loss: 769.0543 - val_mse: 769.0543 - val_mae: 15.3487 - val_rmse: 15.3487 - val_mape: 179.8793 - val_msle: 0.8165
Epoch 4/100
2341/2341 [==============================] - 14s 6ms/step - loss: 792.6050 - mse: 792.6050 - mae: 15.6700 - rmse: 15.6700 - mape

Epoch 30/100
2341/2341 [==============================] - 13s 6ms/step - loss: 680.1046 - mse: 680.1046 - mae: 14.1653 - rmse: 14.1653 - mape: 177.4539 - msle: 0.8035 - val_loss: 765.4863 - val_mse: 765.4863 - val_mae: 14.9709 - val_rmse: 14.9709 - val_mape: 175.4105 - val_msle: 0.8041
Epoch 31/100
2341/2341 [==============================] - 14s 6ms/step - loss: 676.1561 - mse: 676.1561 - mae: 14.1022 - rmse: 14.1022 - mape: 175.0919 - msle: 0.7947 - val_loss: 767.3024 - val_mse: 767.3024 - val_mae: 14.3301 - val_rmse: 14.3301 - val_mape: 141.6830 - val_msle: 0.6995
Epoch 32/100
2341/2341 [==============================] - 14s 6ms/step - loss: 673.3386 - mse: 673.3386 - mae: 14.0925 - rmse: 14.0925 - mape: 173.3364 - msle: 0.7876 - val_loss: 757.4036 - val_mse: 757.4036 - val_mae: 14.9711 - val_rmse: 14.9711 - val_mape: 179.1418 - val_msle: 0.8163
Epoch 33/100
2341/2341 [==============================] - 14s 6ms/step - loss: 669.7418 - mse: 669.7418 - mae: 14.0351 - rmse: 14.0351 - ma

2341/2341 [==============================] - 17s 7ms/step - loss: 609.7729 - mse: 609.7729 - mae: 13.3436 - rmse: 13.3436 - mape: 169.1893 - msle: 0.7625 - val_loss: 803.0488 - val_mse: 803.0488 - val_mae: 14.6795 - val_rmse: 14.6795 - val_mape: 158.8362 - val_msle: 0.7567
Epoch 59/100
 410/2341 [====>.........................] - ETA: 11s - loss: 611.6042 - mse: 611.6042 - mae: 13.4027 - rmse: 13.4027 - mape: 167.7480 - msle: 0.7608

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['mse'], color = '#4472C4')
plt.plot(history.history['val_mse'], color = '#ED7D31')
plt.title('Model MSE')
plt.ylabel('MSE')
#plt.ylim(0.7,0.98)
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig('results/DNN_MSE_Reg.png')
plt.show()

In [ ]:
val_loss_per_epoch = history.history['val_mse']
best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

In [ ]:
start_time = time.time()
hypermodel = tuner.hypermodel.build(best_hps)
print(str(model),"--- %s seconds ---" % ((time.time() - start_time)))

# Retrain the model
history = hypermodel.fit(X_train, y_train_reg,validation_split=0.33, epochs=best_epoch)

In [ ]:
# Print Model Layout:
plot_model(hypermodel, to_file='hypermodel_reg.png', show_shapes=True)

In [ ]:
eval_result = hypermodel.evaluate(X_test, y_test_reg)
print("[loss, MSE, MAE, RMSE, MAPE, MSLE]:", eval_result)

# 10-Fold Cross-Validation

In [ ]:
# Define the K-fold Cross Validator

num_folds = 10
kfold = KFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
loss_per_fold = []
MAE_per_fold = []

inputs = np.concatenate((X_train, X_test), axis=0)
targets = np.concatenate((y_train_reg, y_test_reg), axis=0)

for train, test in kfold.split(inputs, targets):

    best_model = tuner.hypermodel.build(best_hps)

    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    best_model.fit(inputs[train], targets[train], epochs=best_epoch, verbose=0, callbacks = [stop_early])
    score = model.evaluate(inputs[test], targets[test], verbose=0)
    print(f'Score for fold {fold_no}: {score}')

    yhat = model.predict(inputs[test])
    print('MAE: %.3f' % mean_absolute_error(targets[test], yhat))

    MAE_per_fold.append(mean_absolute_error(targets[test], yhat))
    loss_per_fold.append(score)

    # Increase fold number
    fold_no = fold_no + 1

In [ ]:
loss = []
mse = []
mae = []
rmse = []
mape = []
msle = []
for i in range(len(loss_per_fold)):
    loss.append(loss_per_fold[i][0])
    mse.append(loss_per_fold[i][1])
    mae.append(loss_per_fold[i][2])
    rmse.append(loss_per_fold[i][3])
    mape.append(loss_per_fold[i][4])
    msle.append(loss_per_fold[i][5])

In [ ]:
print("Mean Loss: %s" % statistics.mean(loss))
print("Mean MSE: %s" % statistics.mean(mse))
print("Mean MAE: %s" % statistics.mean(mae))
print("Mean RMSE: %s" % statistics.mean(rmse))
print("Mean MAPE: %s" % statistics.mean(mape))
print("Mean MSLE: %s" % statistics.mean(msle))

In [ ]:
print("SD Loss: %s" % statistics.stdev(loss))
print("SD MSE: %s" % statistics.stdev(mse))
print("SD MAE: %s" % statistics.stdev(mae))
print("SD RMSE: %s" % statistics.stdev(rmse))
print("SD MAPE: %s" % statistics.stdev(mape))
print("SD MSLE: %s" % statistics.stdev(msle))

# Plot Results

In [ ]:
plt.figure(figsize=(20, 8))
#Create the plots
plt.plot(range(1,len(loss)+1),loss, color = '#4472C4', label = 'MSE')
#plt.plot(range(1,len(mape)+1),mape, color='red', label = 'MAPE')
#Create the title, axis description and legend
#plt.title('Results KFOLD Cross-Validation')
plt.ylabel('MSE')
plt.xlabel("Numer of Fold")
#plt.legend()
#plt.grid()
plt.xticks(rotation = 0)
plt.savefig('results/DNN_KFold_MSE.png')
plt.rcParams.update({'font.size': 12})

In [ ]:
plt.figure(figsize=(20, 8))
#Create the plots
#plt.plot(range(1,len(loss)+1),loss, color='blue', label = 'MSE')
plt.plot(range(1,len(mape)+1),mape, color = '#4472C4', label = 'MAPE')
#Create the title, axis description and legend
#plt.title('Results KFOLD Cross-Validation')
plt.ylabel('MAPE')
plt.xlabel("Numer of Fold")
#plt.legend()
#plt.grid()
plt.xticks(rotation = 0)
plt.savefig('results/DNN_KFold_MAPE.png')
plt.rcParams.update({'font.size': 12})

In [ ]:
plt.figure(figsize=(20, 8))
#Create the plots
#plt.plot(range(1,len(MAE_per_fold)+1),MAE_per_fold, color='blue', label = 'MAE')
plt.plot(range(1,len(msle)+1),msle, color = '#4472C4', label = 'MSLE')
#Create the title, axis description and legend
#plt.title('Results KFOLD Cross-Validation')
plt.ylabel('MSLE')
plt.xlabel("Numer of Fold")
#plt.legend()
#plt.grid()
plt.xticks(rotation = 0)
plt.savefig('results/DNN_KFold_MSLE.png')
plt.rcParams.update({'font.size': 12})